In [47]:
import os

In [48]:
os.chdir("C:\\Users\\kevin Umraliya\\OneDrive\\Desktop\\project\\Chicken-Disease-Classification")
%pwd

'C:\\Users\\kevin Umraliya\\OneDrive\\Desktop\\project\\Chicken-Disease-Classification'

In [49]:
import tensorflow as tf

In [50]:
model = tf.keras.models.load_model("artifacts/training/trained_model.h5")

In [51]:
from dataclasses import dataclass
from pathlib import Path   

@dataclass(frozen=True)
class EvaluationConfig:
    path_to_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [52]:
from ChickenDiseaseClassification.constants import *
from ChickenDiseaseClassification.utils.common import read_yaml, create_directories

In [53]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARMAS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_to_model = "artifacts/training/trained_model.h5",  
            training_data = "artifacts/data_ingestion/data",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config


In [54]:
from urllib.parse import urlparse

In [55]:
from ChickenDiseaseClassification.utils.common import save_json


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs 
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_to_model)
        self._valid_generator()
        self.score = self.model.evaluate(
            self.valid_generator,
        )

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [56]:
try:
    config = ConfigurationManager()
    val_config = config.get_evaluation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

Found 233 images belonging to 2 classes.
15/15 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.9365 - loss: 0.1887
